# Explore here

In [99]:
import requests
import pandas as pd

url= "https://api.worldbank.org/v2/indicator"
params = {
    "format" : "json",
    "per_page": 200,
}

response = requests.get(url, params=params, timeout=30)
response.raise_for_status() #similar to try catch error
payload = response.json()
print(payload[1][0])
print(type(payload[1][0]))



{'id': '1.0.HCount.1.90usd', 'name': 'Poverty Headcount ($1.90 a day)', 'unit': '', 'source': {'id': '37', 'value': 'LAC Equity Lab'}, 'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2011 PPP) below the poverty line.', 'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).', 'topics': [{'id': '11', 'value': 'Poverty '}]}
<class 'dict'>


The goal is to analyze the socioeconomic and environmental evolution of 5 countries chosen by you between 2010 and 2024.

Countries: Spain, Mexico, Venezuela, Argentina, Colombia
Indicators: GDP, Trade, Exports, GINI, Tobacco, Alcohol

In [100]:
indicators_db = payload[1]
# print("indicators_db:", indicators_db)
indicator_list = []

for indicator in indicators_db:
    # print(f"Indicator ID: {indicator['id']}, Name: {indicator['name']}")
    indicator_list.append({"id": indicator['id'], "name": indicator['name']})
    

# for key in indicators_db.keys():
#     print(f"key: {key} || value: {indicators_db[key]}")
#     if key == "id" or key == "name":
#         indicator_list.append(indicators_db[key]) 

print("indicator_list:", indicator_list)


indicator_list: [{'id': '1.0.HCount.1.90usd', 'name': 'Poverty Headcount ($1.90 a day)'}, {'id': '1.0.HCount.2.5usd', 'name': 'Poverty Headcount ($2.50 a day)'}, {'id': '1.0.HCount.Mid10to50', 'name': 'Middle Class ($10-50 a day) Headcount'}, {'id': '1.0.HCount.Ofcl', 'name': 'Official Moderate Poverty Rate-National'}, {'id': '1.0.HCount.Poor4uds', 'name': 'Poverty Headcount ($4 a day)'}, {'id': '1.0.HCount.Vul4to10', 'name': 'Vulnerable ($4-10 a day) Headcount'}, {'id': '1.0.PGap.1.90usd', 'name': 'Poverty Gap ($1.90 a day)'}, {'id': '1.0.PGap.2.5usd', 'name': 'Poverty Gap ($2.50 a day)'}, {'id': '1.0.PGap.Poor4uds', 'name': 'Poverty Gap ($4 a day)'}, {'id': '1.0.PSev.1.90usd', 'name': 'Poverty Severity ($1.90 a day)'}, {'id': '1.0.PSev.2.5usd', 'name': 'Poverty Severity ($2.50 a day)'}, {'id': '1.0.PSev.Poor4uds', 'name': 'Poverty Severity ($4 a day)'}, {'id': '1.1.HCount.1.90usd', 'name': 'Poverty Headcount ($1.90 a day)-Rural'}, {'id': '1.1.HCount.2.5usd', 'name': 'Poverty Headcoun

In [120]:
df_indicators = pd.DataFrame(indicator_list)
# print(df_indicators.head())
column = df_indicators["name"]
gdp_index = []
for index, name in enumerate(column):
    # print(f"index: {index} || name: {name}")
    if "GROSS DOMESTIC PRODUCT" in name:
        print(f"Found GROSS DOMESTIC PRODUCT at index: {index}")
        gdp_index.append({"index": index, "name": column[index]})

print("gdp_index:", gdp_index)
    

Found GROSS DOMESTIC PRODUCT at index: 41
Found GROSS DOMESTIC PRODUCT at index: 42
gdp_index: [{'index': 41, 'name': '100000:GROSS DOMESTIC PRODUCT'}, {'index': 42, 'name': '1000000:GROSS DOMESTIC PRODUCT'}]
